In [1]:
import pandas as pd
import pathlib as Path
import requests
from dotenv import load_dotenv
import os 
import pyfolio as py
import yfinance as yf
import alpaca_trade_api as tradeapi
import numpy as np
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from MCForecastTools import MCSimulation
import hvplot.pandas
%matplotlib inline

C:\Users\jacob\anaconda3\Lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(
C:\Users\jacob\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
load_dotenv('env.txt')

True

In [3]:
alpaca_api_key = os.getenv("alpaca_api_key")
secret_alpaca_api_key = os.getenv("alpaca_api_key_secret")
api = tradeapi.REST(
    alpaca_api_key,
    secret_alpaca_api_key,
    api_version = "v2"
)

In [19]:
tickers = ["KO", "PFE", "ABT", "NEE", "DUK", "SO", "PG", "COST", "WMT"]
timeframe= "1Day"
start_date= pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()
end_date= pd.Timestamp("2023-11-07", tz="America/New_York").isoformat()
ticker_data = api.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

ticker_data.head()

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2020-01-02 05:00:00+00:00,86.95,86.96,85.8000,35023,86.06,5489065,86.598047,ABT
2020-01-03 05:00:00+00:00,85.89,86.67,85.5900,33223,85.75,5624735,86.070379,ABT
2020-01-06 05:00:00+00:00,86.34,86.36,85.2050,38302,85.62,6558037,85.926388,ABT
2020-01-07 05:00:00+00:00,85.86,86.30,85.5400,30237,85.77,3750700,85.916077,ABT
2020-01-08 05:00:00+00:00,86.21,86.66,85.9285,41290,86.20,6516204,86.264174,ABT


In [20]:
KO = ticker_data[ticker_data["symbol"]=="KO"].drop("symbol", axis=1)
PFE = ticker_data[ticker_data["symbol"]=="PFE"].drop("symbol", axis=1)
ABT = ticker_data[ticker_data["symbol"]=="ABT"].drop("symbol", axis=1)
NEE = ticker_data[ticker_data["symbol"]=="NEE"].drop("symbol", axis=1)
DUK = ticker_data[ticker_data["symbol"]=="DUK"].drop("symbol", axis=1)
SO = ticker_data[ticker_data["symbol"]=="SO"].drop("symbol", axis=1)
PG = ticker_data[ticker_data["symbol"]=="PG"].drop("symbol", axis=1)
COST = ticker_data[ticker_data["symbol"]=="COST"].drop("symbol", axis=1)
WMT = ticker_data[ticker_data["symbol"]=="WMT"].drop("symbol", axis=1)

ticker_data = pd.concat([KO, PFE, ABT, NEE, DUK, SO, PG, COST, WMT], axis=1, keys=["KO", "PFE", "ABT", "NEE", "DUK", "SO", "PG", "COST", "WMT"])

ticker_data.head()

KO                                              \
                           close    high    low trade_count   open    volume   
timestamp                                                                      
2020-01-02 05:00:00+00:00  54.99  55.430  54.76       57649  55.32  13777083   
2020-01-03 05:00:00+00:00  54.69  54.990  54.09       52928  54.32  13848262   
2020-01-06 05:00:00+00:00  54.67  54.905  54.52       57245  54.65  17884730   
2020-01-07 05:00:00+00:00  54.25  54.600  54.15       53892  54.45  12324851   
2020-01-08 05:00:00+00:00  54.35  54.640  54.15       49533  54.27  14163579   

                                        PFE                 ...    COST  \
                                vwap  close   high     low  ...    open   
timestamp                                                   ...           
2020-01-02 05:00:00+00:00  55.025146  39.14  39.35  38.875  ...  294.06   
2020-01-03 05:00:00+00:00  54.705894  38.93  39.24  38.670  ...  290.05   
2020-01-06 05:00:00+00:00  54.666738  38.88  39.00  38.700  ...  290.55   
2020-01-07 05:00:00+00:00  54.281430  38.75  39.13  38.680  ...  291.32   
2020-01-08 05:00:00+00:00  54.378579  39.06  39.22  38.750  ...  290.99   

                                                   WMT                      \
                            volume        vwap   close      high       low   
timestamp                                                                    
2020-01-02 05:00:00+00:00  2250899  292.045204  118.94  119.8900  118.7016   
2020-01-03 05:00:00+00:00  2249472  291.416233  117.89  118.7886  117.5900   
2020-01-06 05:00:00+00:00  2989321  291.128319  117.65  118.0900  116.7700   
2020-01-07 05:00:00+00:00  2200602  290.623687  116.56  117.5150  116.2000   
2020-01-08 05:00:00+00:00  2963285  294.065452  116.16  116.7300  115.6800   

                                                                    
                          trade_count    open   volume        vwap  
timestamp                                                           
2020-01-02 05:00:00+00:00       61047  118.91  7499287  119.077035  
2020-01-03 05:00:00+00:00       42178  118.27  6409264  118.043475  
2020-01-06 05:00:00+00:00       44340  117.43  7881155  117.549913  
2020-01-07 05:00:00+00:00       54639  117.26  7870128  116.653749  
2020-01-08 05:00:00+00:00       53485  116.30  6441726  116.186007  

[5 rows x 63 columns]

In [30]:
ticker_data_closing = pd.DataFrame()

for ticker in tickers:
    ticker_data_closing[ticker] = ticker_data[ticker]["close"]

ticker_data_closing.index = ticker_data_closing.index.date

contraction_portfolio_returns = ticker_data_closing.pct_change()
ticker_data_closing.head()

,KO,PFE,ABT,NEE,DUK,SO,PG,COST,WMT
2020-01-02,54.99,39.14,86.95,238.62,90.34,62.62,123.41,291.49,118.94
2020-01-03,54.69,38.93,85.89,240.32,90.40,62.56,122.58,291.73,117.89
2020-01-06,54.67,38.88,86.34,241.52,90.84,62.81,122.75,291.81,117.65
2020-01-07,54.25,38.75,85.86,241.31,90.27,62.62,121.99,291.35,116.56
2020-01-08,54.35,39.06,86.21,241.20,90.40,62.60,122.51,294.69,116.16


In [7]:
ticker_data_closing.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [31]:
# 30 day rolling average

# line chat
contraction_portfolio_returns.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [12]:
tickers_1 = ["AMZN", "TSLA", "MCD", "TM", "MSFT", "AAPL", "GOOGL", "JPM", "BAC", "TRST"]
timeframe= "1Day"
start_date_1= pd.Timestamp("2017-01-01", tz="America/New_York").isoformat()
end_date_1= pd.Timestamp("2019-12-31", tz="America/New_York").isoformat()
ticker_data_1 = api.get_bars(
    tickers_1,
    timeframe,
    start=start_date_1,
    end=end_date_1
).df

ticker_data_1.head()

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2017-01-03 05:00:00+00:00,116.15,116.3300,114.76,158508,115.75,31713319,115.655776,AAPL
2017-01-04 05:00:00+00:00,116.02,116.5100,115.75,129948,115.90,23234449,116.120635,AAPL
2017-01-05 05:00:00+00:00,116.61,116.8642,115.81,136223,115.97,23572537,116.458819,AAPL
2017-01-06 05:00:00+00:00,117.91,118.1600,116.47,177799,116.75,33913959,117.732493,AAPL
2017-01-09 05:00:00+00:00,118.99,119.4300,117.94,184632,118.00,36262611,118.969146,AAPL


In [13]:
# Create and concat df
amzn = ticker_data_1[ticker_data_1["symbol"]=="AMZN"].drop("symbol", axis=1)
tsla = ticker_data_1[ticker_data_1["symbol"]=="TSLA"].drop("symbol", axis=1)
mcd = ticker_data_1[ticker_data_1["symbol"]=="MCD"].drop("symbol", axis=1)
tm = ticker_data_1[ticker_data_1["symbol"]=="TM"].drop("symbol", axis=1)
msft = ticker_data_1[ticker_data_1["symbol"]=="MSFT"].drop("symbol", axis=1)
aapl = ticker_data_1[ticker_data_1["symbol"]=="AAPL"].drop("symbol", axis=1)
googl = ticker_data_1[ticker_data_1["symbol"]=="GOOGL"].drop("symbol", axis=1)
jpm = ticker_data_1[ticker_data_1["symbol"]=="JPM"].drop("symbol", axis=1)
bac = ticker_data_1[ticker_data_1["symbol"]=="BAC"].drop("symbol", axis=1)
trst = ticker_data_1[ticker_data_1["symbol"]=="TRST"].drop("symbol", axis=1)

ticker_data_1 = pd.concat([amzn, tsla, mcd, tm, msft, aapl, googl, jpm, bac, trst], axis=1, keys=["AMZN", "TSLA", "MCD", "TM", "MSFT", "AAPL", "GOOGL", "JPM", "BAC", "TRST"])

In [27]:
ticker_data_closing_1 = pd.DataFrame()

for ticker in tickers_1:
    ticker_data_closing_1[ticker] = ticker_data_1[ticker]["close"]

ticker_data_closing_1.index = ticker_data_closing_1.index.date

expansion_portfolio_returns = ticker_data_closing_1.pct_change()
expansion_portfolio_returns = expansion_portfolio_returns.dropna()
ticker_data_closing_1.head()

,AMZN,TSLA,MCD,TM,MSFT,AAPL,GOOGL,JPM,BAC,TRST
2017-01-03,753.67,216.99,119.62,118.55,62.58,116.15,808.01,87.23,22.53,8.85
2017-01-04,757.18,226.99,119.48,121.19,62.30,116.02,807.77,86.91,22.95,8.95
2017-01-05,780.45,226.75,119.70,120.44,62.30,116.61,813.02,86.11,22.68,8.70
2017-01-06,795.99,229.01,120.76,120.13,62.84,117.91,825.21,86.12,22.68,8.65
2017-01-09,796.92,231.28,120.43,119.74,62.64,118.99,827.18,86.18,22.55,8.50


In [17]:
ticker_data_closing_1.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [29]:
# 30 day rolling average

# line chart
expansion_portfolio_returns.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [ ]:
# analysis on stock versus SPDR fund (SPX as benchmark)
